# 🤖 Joke AI Bot with LangGraph

An interactive joke bot built with **LangGraph** and **OpenAI GPT-4o**.

The bot follows a stateful graph workflow:
- **Greet User** → Welcome the user and ask if they want a joke
- **Get & Check Topic** → Get a topic from the user and validate it (loops back if invalid)
- **Continue?** → Ask the user if they want another joke
- **Say Bye** → Farewell message with joke count

![Architecture](https://raw.githubusercontent.com/Hazem-Galal/Joke-AI-Bot-LangGraph/main/joke%20bot.png)

## 1. Install Dependencies

In [ ]:
!pip install -q langgraph langchain-openai langchain-core

## 2. Setup API Key

Add your OpenAI API key to **Colab Secrets**:
1. Click the 🔑 icon in the left sidebar
2. Add a new secret named `OPENAI_API_KEY`
3. Paste your OpenAI API key as the value
4. Toggle "Notebook access" ON

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
print("✅ API key loaded successfully!")

## 3. Define the State

The state holds all shared data across nodes in the graph.

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages


class JokeBotState(TypedDict):
    """State schema for the Joke Bot."""
    messages: Annotated[list[AnyMessage], add_messages]  # Conversation history
    topic: str              # Current joke topic
    is_valid_topic: bool    # Whether the topic is valid for jokes
    user_choice: str        # User's choice: 'continue' or 'end'
    joke_count: int         # Number of jokes told


print("✅ State defined!")

## 4. Initialize the LLM

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0.9)

print("✅ LLM initialized (GPT-4o)!")

## 5. Define Node Functions

Each node is a Python function that receives the current state, performs logic, and returns state updates.

| Node | Description |
|------|-------------|
| **Greet User** | Welcomes the user and asks if they want a joke |
| **Get & Check Topic** | Prompts for a topic, validates it, and generates a joke |
| **Continue?** | Asks if the user wants another joke |
| **Say Bye** | Farewell message with total joke count |

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage


# ──────────────────────────────────────────────
# Node 1: Greet User
# ──────────────────────────────────────────────
def greet_user(state: JokeBotState) -> dict:
    """Welcome the user and ask if they want to hear a joke."""
    print("\n" + "=" * 50)
    print("🎭  Welcome to the Joke AI Bot!")
    print("    Powered by LangGraph & GPT-4o")
    print("=" * 50)
    print("\nI can tell you jokes on any topic you like!")

    choice = input("\nWould you like to hear a joke? (yes/no): ").strip().lower()
    user_choice = "continue" if choice in ["yes", "y", "sure", "ok", "yeah"] else "end"

    return {
        "user_choice": user_choice,
        "messages": [AIMessage(content="Welcome to the Joke AI Bot! 🎭")],
    }


# ──────────────────────────────────────────────
# Node 2: Get & Check Topic
# ──────────────────────────────────────────────
def get_and_check_topic(state: JokeBotState) -> dict:
    """Get a topic from the user, validate it, and generate a joke if valid."""
    topic = input("\n🎯 Enter a topic for your joke: ").strip()

    if not topic:
        print("❌ You didn't enter a topic. Please try again.")
        return {
            "topic": "",
            "is_valid_topic": False,
        }

    # Ask the LLM to validate the topic and generate a joke
    validation_prompt = [
        SystemMessage(content=(
            "You are a helpful joke validation and generation assistant. "
            "The user will provide a topic. First, determine if the topic is appropriate "
            "and suitable for a joke (not offensive, not nonsensical gibberish). "
            "If the topic is VALID: respond with 'VALID:' followed by a short, clever, "
            "and funny joke about that topic. "
            "If the topic is INVALID: respond with 'INVALID:' followed by a brief "
            "explanation of why it's not suitable."
        )),
        HumanMessage(content=f"Topic: {topic}"),
    ]

    response = llm.invoke(validation_prompt)
    response_text = response.content

    if response_text.upper().startswith("VALID:"):
        joke = response_text[6:].strip()
        print(f"\n😂 Here's your joke about '{topic}':\n")
        print(f"   {joke}")
        print()
        return {
            "topic": topic,
            "is_valid_topic": True,
            "joke_count": state.get("joke_count", 0) + 1,
            "messages": [
                HumanMessage(content=f"Tell me a joke about: {topic}"),
                AIMessage(content=joke),
            ],
        }
    else:
        reason = response_text[8:].strip() if response_text.upper().startswith("INVALID:") else response_text
        print(f"\n❌ That topic isn't great for a joke: {reason}")
        print("   Please try a different topic.\n")
        return {
            "topic": topic,
            "is_valid_topic": False,
        }


# ──────────────────────────────────────────────
# Node 3: Continue?
# ──────────────────────────────────────────────
def continue_prompt(state: JokeBotState) -> dict:
    """Ask the user if they want another joke."""
    joke_count = state.get("joke_count", 0)
    print(f"📊 Jokes told so far: {joke_count}")

    choice = input("\nWould you like another joke? (yes/no): ").strip().lower()
    user_choice = "continue" if choice in ["yes", "y", "sure", "ok", "yeah", "more"] else "end"

    return {"user_choice": user_choice}


# ──────────────────────────────────────────────
# Node 4: Say Bye
# ──────────────────────────────────────────────
def say_bye(state: JokeBotState) -> dict:
    """Say goodbye to the user with a summary."""
    joke_count = state.get("joke_count", 0)
    print("\n" + "=" * 50)
    print("👋 Thanks for using the Joke AI Bot!")
    print(f"   I told you {joke_count} joke(s) today.")
    if joke_count > 0:
        print("   Hope I made you laugh! 😄")
    else:
        print("   Maybe next time! 😊")
    print("=" * 50 + "\n")

    return {
        "messages": [AIMessage(content=f"Goodbye! I told you {joke_count} joke(s). See you next time! 👋")],
    }


print("✅ All node functions defined!")

## 6. Define Routing Functions

Routing functions determine which node to execute next based on the current state.

| After Node | Condition | Next Node |
|---|---|---|
| Greet User | `continue` | Get & Check Topic |
| Greet User | `end` | Say Bye |
| Get & Check Topic | Valid topic | Continue? |
| Get & Check Topic | Invalid topic | Get & Check Topic (self-loop) |
| Get & Check Topic | `end` | Say Bye |
| Continue? | `continue` | Get & Check Topic |
| Continue? | `end` | Say Bye |

In [ ]:
def route_after_greet(state: JokeBotState) -> str:
    """Route after greeting: continue to get topic or say bye."""
    if state.get("user_choice") == "continue":
        return "get_and_check_topic"
    return "say_bye"


def route_after_topic(state: JokeBotState) -> str:
    """Route after topic check: loop back if invalid, continue if valid, or end."""
    if state.get("user_choice") == "end":
        return "say_bye"
    if not state.get("is_valid_topic", False):
        return "get_and_check_topic"  # Self-loop: invalid topic
    return "continue_prompt"  # Valid topic → ask to continue


def route_after_continue(state: JokeBotState) -> str:
    """Route after continue prompt: get another topic or say bye."""
    if state.get("user_choice") == "continue":
        return "get_and_check_topic"
    return "say_bye"


print("✅ Routing functions defined!")

## 7. Build & Compile the Graph

Connect all nodes and edges to form the complete workflow.

In [ ]:
from langgraph.graph import StateGraph, START, END

# Create the graph
workflow = StateGraph(JokeBotState)

# Add nodes
workflow.add_node("greet_user", greet_user)
workflow.add_node("get_and_check_topic", get_and_check_topic)
workflow.add_node("continue_prompt", continue_prompt)
workflow.add_node("say_bye", say_bye)

# Add edges
workflow.add_edge(START, "greet_user")

workflow.add_conditional_edges(
    "greet_user",
    route_after_greet,
    {
        "get_and_check_topic": "get_and_check_topic",
        "say_bye": "say_bye",
    },
)

workflow.add_conditional_edges(
    "get_and_check_topic",
    route_after_topic,
    {
        "get_and_check_topic": "get_and_check_topic",  # Self-loop (invalid topic)
        "continue_prompt": "continue_prompt",           # Valid topic
        "say_bye": "say_bye",                           # End
    },
)

workflow.add_conditional_edges(
    "continue_prompt",
    route_after_continue,
    {
        "get_and_check_topic": "get_and_check_topic",  # Another joke
        "say_bye": "say_bye",                           # End
    },
)

workflow.add_edge("say_bye", END)

# Compile the graph
graph = workflow.compile()

print("✅ Graph built and compiled successfully!")

## 8. Visualize the Graph

Render the graph to verify it matches the intended architecture.

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"⚠️ Could not render graph image: {e}")
    print("\nGraph structure (Mermaid format):")
    print(graph.get_graph().draw_mermaid())

## 9. Run the Bot 🚀

Start the interactive joke bot! Follow the prompts to get jokes.

In [ ]:
# Initial state
initial_state = {
    "messages": [],
    "topic": "",
    "is_valid_topic": False,
    "user_choice": "",
    "joke_count": 0,
}

# Run the graph
final_state = graph.invoke(initial_state, config={"recursion_limit": 100})

print("\n📜 Conversation Summary:")
print("-" * 40)
for msg in final_state["messages"]:
    role = "🤖 Bot" if isinstance(msg, AIMessage) else "👤 You"
    print(f"{role}: {msg.content}")
print("-" * 40)
print(f"Total jokes: {final_state['joke_count']}")